# Calculating Views, Visits and Users

In [0]:
CREATE OR REPLACE VIEW dev.bi.content_visitors_monthly_history_mgtv as
select 
date_trunc('month',pvf.page_view_dt) as page_view_month,
coalesce(pvf.course_id,pvf.phoenix_specialization_id) as content_id,
underlying_content_type as primary_slug_type,
count(*) as total_views,
count(distinct try_cast(pvf.user_id as string) || pvf.cookie_id) as total_visits, 
count(distinct coalesce(try_cast(pvf.user_id as string), pvf.cookie_id)) as total_users
from prod.gold.page_views_v2 pvf
inner join dev.bi.content_dim_kpi b on coalesce(pvf.course_id,pvf.phoenix_specialization_id) = b.underlying_product_item_id
WHERE date(pvf.page_view_dt) < to_date(date_trunc('month',current_date))
and date(pvf.page_view_dt) >=to_date(date_trunc('month', add_months(current_date, -1)))
group by 1,2,3
;

In [0]:
DELETE FROM dev.bi.content_visitors_monthly_history_mgtf
WHERE page_view_month >= to_date(date_trunc('month', add_months(current_date, -1)))

num_affected_rows
28093


In [0]:
INSERT INTO dev.bi.content_visitors_monthly_history_mgtf
select * from dev.bi.content_visitors_monthly_history_mgtv

num_affected_rows,num_inserted_rows
28092,28092


## Final view for views , visits and users

In [0]:
create or replace view dev.bi.content_visitors_monthly_history_view_mgtv as
with 
totf_agg (select date(page_view_month) as dt_month
          ,content_sub_domain
          ,cluster_rank
          ,SUM(total_views) as total_views
          ,SUM(total_visits) as total_visits
          ,SUM(total_users) as total_users
          from dev.bi.content_visitors_monthly_history_mgtf a
          left join dev.bi.content_dim_kpi b on a.content_id = b.underlying_product_item_id
          left join dev.bi.s12n_partner_cluster_ranks_mgtv c on b.partner_id = c.partner_id
          where primary_slug_type = 's12n' and content_sub_domain is not null
          group by 1,2,3)
,totf_view (select dt_month,content_sub_domain,cluster_rank,
          row_number() OVER (PARTITION BY content_sub_domain,cluster_rank ORDER BY dt_month) AS row_num, 
          SUM(total_views)  OVER (PARTITION BY content_sub_domain,cluster_rank ORDER BY dt_month ROWS BETWEEN 12 PRECEDING AND 1 PRECEDING) AS rolling_n12m_views,
          SUM(total_visits) OVER (PARTITION BY content_sub_domain,cluster_rank ORDER BY dt_month ROWS BETWEEN 12 PRECEDING AND 1 PRECEDING) AS rolling_n12m_visits,
          SUM(total_users)  OVER (PARTITION BY content_sub_domain,cluster_rank ORDER BY dt_month ROWS BETWEEN 12 PRECEDING AND 1 PRECEDING) AS rolling_n12m_users
          from totf_agg)
select dt_month,
lower(replace(content_sub_domain,' ','-')) as content_sub_domain,
cluster_rank,
sum(rolling_n12m_views) as sd_avg_rolling_n12m_views,
sum(rolling_n12m_visits) as sd_avg_rolling_n12m_visits,
sum(rolling_n12m_users) as sd_avg_rolling_n12m_users
from totf_view
group by 1,2,3;

# Calculating Impressions

In [0]:
CREATE OR REPLACE VIEW dev.bi.content_impressions_monthly_history_mgtv as
SELECT date_trunc('month',impression_date) as dt_month,
      entity_id,
      split_part(entity_id, '~', 1) as content_type,
      -- split_part(entity_id, '~', 2) as content_id,
      COUNT(DISTINCT event_session) AS impression_count
      FROM prod.gold_base.impressions 
      WHERE date(impression_date) < to_date(date_trunc('month',current_date))
        and date(impression_date) >=to_date(date_trunc('month', add_months(current_date, -5)))
GROUP BY 1,2,3
having content_type in ('course','s12n','certificate')

In [0]:
DELETE FROM dev.bi.content_impressions_monthly_history_mgtf
WHERE dt_month >= to_date(date_trunc('month', add_months(current_date, -5)))

num_affected_rows
108868


In [0]:
INSERT INTO dev.bi.content_impressions_monthly_history_mgtf
select * from dev.bi.content_impressions_monthly_history_mgtv

num_affected_rows,num_inserted_rows
108868,108868


# Final view for Impressions

In [0]:
create or replace view dev.bi.content_impressions_monthly_history_view_mgtv as 
with imprsn_view (select dt_month
                  ,case when len(split_part(entity_id, '~', 3)) > 0 then split_part(entity_id, '~', 3) else split_part(entity_id, '~', 2) end as content_id
                  ,SUM(impression_count) as impression_count
                  from dev.bi.content_impressions_monthly_history 
                  where content_type = 's12n' 
                  group by 1,2)
,imprsn_agg (select  date(dt_month) as dt_month
            ,content_sub_domain
            ,cluster_rank
            ,SUM(impression_count) as impression_count
            from imprsn_view a
            inner join dev.bi.content_dim_kpi b on a.content_id = b.underlying_product_item_id
            left join dev.bi.s12n_partner_cluster_ranks_mgtv c on b.partner_id = c.partner_id
            where content_sub_domain is not null
            group by 1,2,3)
,imprsn_final (select dt_month
            ,content_sub_domain
            ,cluster_rank
            ,row_number() OVER (PARTITION BY content_sub_domain,cluster_rank ORDER BY dt_month) AS row_num
            ,SUM(impression_count)  OVER (PARTITION BY content_sub_domain,cluster_rank ORDER BY dt_month ROWS BETWEEN 12 PRECEDING AND 1 PRECEDING) AS rolling_n12m_impressions
            from imprsn_agg)
select dt_month,
lower(replace(content_sub_domain,' ','-')) as content_sub_domain,
cluster_rank,
sum(rolling_n12m_impressions) as sd_avg_rolling_n12m_impressions
from imprsn_final
group by 1,2,3;